In [1]:
import numpy as np
import pandas as pd

In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
df =individual_household_electric_power_consumption = fetch_ucirepo(id=235) 
  
# data (as pandas dataframes) 
X = individual_household_electric_power_consumption.data.features 
y = individual_household_electric_power_consumption.data.targets 
  
# metadata 
print(individual_household_electric_power_consumption.metadata) 
  
# variable information 
print(individual_household_electric_power_consumption.variables) 


C:\Users\User\AppData\Roaming\Python\Python39\site-packages\ucimlrepo\fetch.py:97: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


{'uci_id': 235, 'name': 'Individual Household Electric Power Consumption', 'repository_url': 'https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption', 'data_url': 'https://archive.ics.uci.edu/static/public/235/data.csv', 'abstract': 'Measurements of electric power consumption in one household with a one-minute sampling rate over a period of almost 4 years. Different electrical quantities and some sub-metering values are available.', 'area': 'Physics and Chemistry', 'tasks': ['Regression', 'Clustering'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 2075259, 'num_features': 9, 'feature_types': ['Real'], 'demographics': [], 'target_col': None, 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2006, 'last_updated': 'Fri Mar 08 2024', 'dataset_doi': '10.24432/C58K54', 'creators': ['Georges Hebrail', 'Alice Berard'], 'intro_paper': None, 'additional_info': {'summary': 'This archiv

In [3]:
df = pd.concat([X, y], axis=1)
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [4]:
#12

df.replace('?', np.nan, inplace=True)
df.dropna(inplace=True)

# Convert columns to appropriate data types
df['Global_active_power'] = df['Global_active_power'].astype(float)
df['Voltage'] = df['Voltage'].astype(float)

# Calculate the Pearson correlation coefficient
correlation = df['Global_active_power'].corr(df['Voltage'])

# Print the correlation coefficient rounded to 3 decimal places
print(f"Pearson correlation coefficient between global_active_power and voltage: {correlation:.3f}")

Pearson correlation coefficient between global_active_power and voltage: -0.400


In [5]:
#13
df['Voltage'] = df['Voltage'].astype(float)
df['Global_intensity'] = df['Global_intensity'].astype(float)

# Calculate the Pearson correlation coefficient
correlation = df['Voltage'].corr(df['Global_intensity'])

# Print the correlation coefficient rounded to 2 decimal places
print(f"{correlation:.2f}")

-0.41


In [6]:
#20
df['Global_active_power'] = df['Global_active_power'].astype(float)

# Aggregate the data by day to get daily sums
df.set_index('datetime', inplace=True)
daily_data = df.resample('D').sum().reset_index()

# Prepare the data for Prophet
daily_data = daily_data[['datetime', 'Global_active_power']].rename(columns={'datetime': 'ds', 'Global_active_power': 'y'})

# Train the model
model = Prophet(yearly_seasonality=True)
model.fit(daily_data)

# Make future dataframe
future = model.make_future_dataframe(periods=365)

KeyError: "None of ['datetime'] are in the columns"

In [7]:
#14
import pandas as pd
from fbprophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error



# Split the data into train and test sets
train_data = daily_data.iloc[:-300]
test_data = daily_data.iloc[-300:]

# Train the model
model = Prophet()
model.fit(train_data)

# Make future dataframe for the test period
future = model.make_future_dataframe(periods=300)
forecast = model.predict(future)

# Calculate MAPE
# Get the predicted values corresponding to the test period
y_true = test_data['y'].values
y_pred = forecast['yhat'][-300:].values

# Calculate MAPE
mape = mean_absolute_percentage_error(y_true, y_pred) * 100

# Print the MAPE rounded to 2 decimal places
print(f"MAPE: {mape:.2f}%")


ModuleNotFoundError: No module named 'fbprophet'

In [8]:
#15
from sklearn.metrics import mean_squared_error

# Convert columns to appropriate data types
df['Global_active_power'] = df['Global_active_power'].astype(float)

# Aggregate the data by day to get daily sums
df.set_index('datetime', inplace=True)
daily_data = df.resample('D').sum().reset_index()

# Prepare the data for Prophet
daily_data = daily_data[['datetime', 'Global_active_power']].rename(columns={'datetime': 'ds', 'Global_active_power': 'y'})

# Split the data into train and test sets
train_data = daily_data.iloc[:-300]
test_data = daily_data.iloc[-300:]

# Train the model
model = Prophet()
model.fit(train_data)

# Make future dataframe for the test period
future = model.make_future_dataframe(periods=300)
forecast = model.predict(future)

# Calculate RMSE
# Get the predicted values corresponding to the test period
y_true = test_data['y'].values
y_pred = forecast['yhat'][-300:].values

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

# Print the RMSE rounded to 2 decimal places
print(f"RMSE: {rmse:.2f}")


KeyError: "None of ['datetime'] are in the columns"

In [9]:
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error

# Load the dataset
data = pd.read_csv('your_dataset.csv', delimiter=';', parse_dates={'ds': ['date', 'time']}, infer_datetime_format=True)
data['ds'] = pd.to_datetime(data['ds'], format='%d/%m/%Y %H:%M')

# Selecting relevant columns
df = data[['ds', 'globalactivepower']].copy()
df.columns = ['ds', 'y']

# Splitting into train and test sets
train = df.iloc[:-300]
test = df.iloc[-300:]

# Initializing Prophet model
model = Prophet()

# Fit the model
model.fit(train)

# Make predictions for the test set
future = model.make_future_dataframe(periods=300, freq='D')
forecast = model.predict(future)

# Extract predicted values and actual values
y_pred = forecast[-300:]['yhat'].values
y_true = test['y'].values

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

# Print RMSE rounded to 2 decimal places
print(f"RMSE: {rmse:.2f}")


ModuleNotFoundError: No module named 'fbprophet'